In [46]:
import pandas as pd
import re

# Main

In [84]:
df_bem_pb = pd.read_csv('bem_candidato_2020/bem_candidato_2020_PB.csv',error_bad_lines=False, encoding='latin-1',delimiter =';')

In [37]:
f = open("bem_candidato_2020/bem_PB_txt", "r", encoding="utf8")
texto = f.readlines()
lst_aux = [re.sub(r'\n',' ',re.sub(r'"', ' ', element)).split(';') for element in texto]
df_pb = pd.DataFrame(lst_aux[1:], columns = lst_aux[0]) 

In [68]:
#lst_aux = []
#for element in texto:
#    element_aux = re.sub(r'"', ' ', element)
#    element_aux = re.sub(r'\n', ' ', element_aux)
#    element_aux = element_aux.split(';')
#    lst_aux.append(element_aux)

In [89]:
df_pb.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,NR_ORDEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULTIMA_ATUALIZACAO,HH_ULTIMA_ATUALIZACAO
0,20/10/2020,07:06:40,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,19666,JOCA CLAUDINO,150000633249,1,24,Bem relacionado com o exercício da atividade ...,TITULAR PROL EQUIPAMENTOS LOGISTICOS EIRELI C...,"200000,00",17/10/2020,18:52:47
1,20/10/2020,07:06:40,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,19666,JOCA CLAUDINO,150000633249,2,63,Dinheiro em espécie - moeda nacional,RENDIMENTOS TRIBUTÁRIOS RECEBIDOS DE PESSOA F...,"28500,00",17/10/2020,18:52:47
2,20/10/2020,07:06:40,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,19666,JOCA CLAUDINO,150000633250,1,63,Dinheiro em espécie - moeda nacional,Rendimentos Tributários Recebidos de Pessoa J...,"2060,64",17/10/2020,17:56:20
3,20/10/2020,07:06:40,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,19666,JOCA CLAUDINO,150000633250,2,63,Dinheiro em espécie - moeda nacional,Rendimentos Tributário Recebidos de Pessoa Ju...,"7065,74",17/10/2020,17:56:20
4,20/10/2020,07:06:40,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,PB,21075,NOVA PALMEIRA,150000633290,1,11,Apartamento,"Apartamento localizado na Rua Espírito Santo,...","90000,00",01/10/2020,17:44:03


In [82]:
df_lista = pd.read_csv('Lista-Candidatos-Vereador-JOÃO PESSOA-2020.csv',delimiter =';')

In [83]:
df_lista.head()

,Nome Completo,Nome Urna,Cargo,Número,Partido,Coligação,Situação,Situação Pós-Pleito
0,ABRAÃO CAVALCANTI DE LACERDA,ABRAÃO CAVALCANTI,Vereador,35123,PMB,PMB,Aguardando julgamento,Concorrendo
1,ADAILTON IZIDRO SANTOS DA SILVA,ADAILTON IRMÃO DOS PNEUS,Vereador,20123,PSC,PSC,Aguardando julgamento,Concorrendo
2,ADEILDE TRIGUEIRO DE ARAÚJO,ADEILDE TRIGUEIRO,Vereador,22077,PL,PL,Aguardando julgamento,Concorrendo
3,ADEILZA DE FARIAS MOREIRA,ADEILZA,Vereador,22111,PL,PL,Aguardando julgamento,Concorrendo
4,ADEMIR DE LUCENA,ADEMIR DE LUCENA,Vereador,77001,SOLIDARIEDADE,SOLIDARIEDADE,Deferido,Concorrendo
